In [1]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = './PlantVillage/'
width=256
height=256
depth=3

In [3]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [4]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing e073380b-e5ad-4e07-8ed5-5adec8bac1ff___NREC_B.Spot 1906.JPG ...
[INFO] Processing e184272e-73aa-4b9c-a03b-1597c761de1f___JR_B.Spot 8938.JPG ...
[INFO] Processing e481606b-2d13-4965-9e0a-44f02a8fa701___JR_B.Spot 8833.JPG ...
[INFO] Processing e5666de8-558c-42cb-b40f-51b81f850aed___NREC_B.Spot 1903.JPG ...
[INFO] Processing e5781299-7fd6-4d80-aca1-d78910d9da1c___JR_B.Spot 3371.JPG ...
[INFO] Processing e825c445-4b5b-4642-8d83-6db23c5d8348___JR_B.Spot 3341.JPG ...
[INFO] Processing e91037f2-392c-45de-a2eb-2abd31ca3089___JR_B.Spot 3267.JPG ...
[INFO] Processing e9714b47-3088-4ef5-9f18-02983c4558fc___JR_B.Spot 3144.JPG ...
[INFO] Processing e973e4ca-1670-4fa7-9206-49046080d897___NREC_B.Spot 1831.JPG ...
[INFO] Processing ea460c0a-532e-492b-93fa-5b9b936c7506___NREC_B.Spot 1810.JPG ...
[INFO] Processing ea679763-149c-44b1-9593-e82242f303cb___NREC_B.Spot 1985.JPG ...
[INFO] Processing ea9d49e5-e3ee-4429-8592-4ce9a910376f___NREC_B.Spot 1821.JPG ...
[I

In [5]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [6]:
print(label_binarizer.classes_)

['09188838-cc89-4925-94c5-d9563c4cb4bf___UF.GRC_YLCV_Lab 02974.JPG'
 '09266523-41f5-42cd-bf15-5f13a6cec876___UF.GRC_YLCV_Lab 02419.JPG'
 '09320877-b96e-46ea-b1d3-6f915d60180f___UF.GRC_YLCV_Lab 02550.JPG'
 '09397385-9a50-4d0a-bc52-bccfd6aa7c59___UF.GRC_YLCV_Lab 01763.JPG'
 '11030437-942c-4cb4-a40e-b35236778a4b___UF.GRC_YLCV_Lab 02923.JPG'
 '11061195-ce9c-4171-819d-663342a186e8___YLCV_NREC 0257.JPG'
 '11796227-f10e-4502-a849-ddff117a083c___UF.GRC_YLCV_Lab 03130.JPG'
 '12313111-b847-4491-934c-f5861e98db88___YLCV_NREC 0272.JPG'
 '13248326-a01f-4f05-8502-fe2ecd936ccc___UF.GRC_YLCV_Lab 02422.JPG'
 '14034780-0670-4484-be42-cb2b12607c40___YLCV_NREC 2898.JPG'
 '14804695-4a72-4be7-aa85-d675f6c6ff02___YLCV_GCREC 2469.JPG'
 '16395848-34c6-4c92-9119-088b5e815e12___YLCV_NREC 2471.JPG'
 '17815141-f5f6-4657-99f7-029b1e179c8e___YLCV_NREC 0100.JPG'
 '18024559-aaa4-4421-a0af-3897c1a1af5d___YLCV_NREC 0087.JPG'
 '18312065-ee12-45f2-b06a-e72132245a71___UF.GRC_YLCV_Lab 09377.JPG'
 '18431298-49c2-480d-8403-ca

In [7]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [8]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)

[INFO] Spliting data to train, test


In [11]:
print(x_train)

[[[[0.649  0.6577 0.6577]
   [0.6445 0.6533 0.6533]
   [0.64   0.649  0.649 ]
   ...
   [0.676  0.6934 0.671 ]
   [0.6665 0.6846 0.662 ]
   [0.6577 0.676  0.6533]]

  [[0.64   0.649  0.649 ]
   [0.64   0.649  0.649 ]
   [0.64   0.649  0.649 ]
   ...
   [0.676  0.6934 0.671 ]
   [0.6665 0.6846 0.662 ]
   [0.662  0.68   0.6577]]

  [[0.6313 0.64   0.64  ]
   [0.6357 0.6445 0.6445]
   [0.64   0.649  0.649 ]
   ...
   [0.68   0.6978 0.676 ]
   [0.676  0.6934 0.671 ]
   [0.671  0.689  0.6665]]

  ...

  [[0.2311 0.2578 0.28  ]
   [0.2356 0.2622 0.2844]
   [0.2356 0.2622 0.2844]
   ...
   [0.2932 0.3157 0.3289]
   [0.2844 0.3066 0.32  ]
   [0.28   0.3022 0.3157]]

  [[0.2489 0.2756 0.2979]
   [0.2489 0.2756 0.2979]
   [0.2444 0.271  0.2932]
   ...
   [0.3066 0.3289 0.3423]
   [0.2932 0.3157 0.3289]
   [0.28   0.3022 0.3157]]

  [[0.2666 0.2932 0.3157]
   [0.2666 0.2932 0.3157]
   [0.2578 0.2844 0.3066]
   ...
   [0.3157 0.338  0.351 ]
   [0.2979 0.32   0.3333]
   [0.2844 0.3066 0.32  ]]]


 

In [12]:
print(x_test)

[[[[0.7334 0.702  0.7334]
   [0.7065 0.676  0.7065]
   [0.676  0.6445 0.676 ]
   ...
   [0.56   0.533  0.582 ]
   [0.5376 0.511  0.56  ]
   [0.4934 0.4666 0.5156]]

  [[0.6846 0.6533 0.6846]
   [0.68   0.649  0.68  ]
   [0.6846 0.6533 0.6846]
   ...
   [0.52   0.4934 0.542 ]
   [0.4888 0.4622 0.511 ]
   [0.449  0.422  0.4712]]

  [[0.662  0.6313 0.662 ]
   [0.6846 0.6533 0.6846]
   [0.7065 0.676  0.7065]
   ...
   [0.5156 0.4888 0.5376]
   [0.5024 0.4756 0.5244]
   [0.4712 0.4443 0.4934]]

  ...

  [[0.6934 0.676  0.7246]
   [0.6577 0.64   0.689 ]
   [0.64   0.622  0.671 ]
   ...
   [0.449  0.44   0.4934]
   [0.4312 0.422  0.4756]
   [0.44   0.4312 0.4844]]

  [[0.68   0.662  0.711 ]
   [0.6665 0.649  0.6978]
   [0.662  0.6445 0.6934]
   ...
   [0.4844 0.4756 0.529 ]
   [0.4578 0.449  0.5024]
   [0.44   0.4312 0.4844]]

  [[0.6665 0.649  0.6978]
   [0.676  0.6577 0.7065]
   [0.689  0.671  0.72  ]
   ...
   [0.4844 0.4756 0.529 ]
   [0.4534 0.4443 0.4978]
   [0.4133 0.4045 0.4578]]]


 

In [16]:
print("X Train - ",len(x_train))
print("X Test - ",len(x_test))

X Train -  6121
X Test -  1531


In [18]:

print("Y Train - ",len(y_train))
print("Y Test - ",len(y_test))

Y Train -  6121
Y Test -  1531
